In [11]:
%%writefile lab/prime_mpi_omp.cpp
#include "mpi.h" 
#include <stdio.h> 
#include <math.h> 
#include <sys/time.h>
#include <omp.h>
#include <iostream>
using namespace std;

//数据量
constexpr long TotalNumStep = 1024;

void exhaust(int* flag_per_rank, int myid, int numprocs, int num_step){
    char machine_name[MPI_MAX_PROCESSOR_NAME]; 
    int namelen;
    int is_cpu=true;
    int* flag_limit = new int[num_step];
    for(int i=1;i<num_step;i++)flag_limit[i]=sqrt(i*numprocs+myid);
    MPI_Get_processor_name(machine_name,&namelen);
    
     #pragma omp target map(from:is_cpu) map(tofrom:flag_per_rank[0:num_step])
    {  
        
        #pragma omp teams distribute parallel for simd
        for (int k=0; k< num_step; k++) {
            if (k==0) is_cpu=omp_is_initial_device();
            //int limit (k*numprocs+myid);
            for(int i=2;i<(k*numprocs+myid);i++){
                if((k*numprocs+myid)%i==0){
                    flag_per_rank[k]=false;
                    break;
                }
            }
        }
    }
    cout << "Rank " << myid << " of " << numprocs
         << " runs on: " << machine_name
         << ", uses device: " << (is_cpu?"CPU":"GPU")
         << "\n";
}

int main(int argc, char* argv[])
{
    int myid, numprocs; 
    struct timeval ts,te;
    MPI_Status status;
    int* flag = new int[TotalNumStep];
    for(int i=0;i<TotalNumStep;i++)flag[i]=true;
    
    MPI_Init(&argc,&argv);
    MPI_Comm_rank(MPI_COMM_WORLD,&myid); 
    MPI_Comm_size(MPI_COMM_WORLD,&numprocs); 
    gettimeofday(&ts,NULL);
    int num_step = TotalNumStep / numprocs;
    int* flag_per_rank = new int[num_step];
    for(int i=0;i<num_step;i++)flag_per_rank[i] = true;
    exhaust(flag_per_rank, myid, numprocs, num_step);
    
    if(myid==0)
    {
        int** all_flag_per_rank = new int*[numprocs];
        all_flag_per_rank[0] = flag_per_rank;
        for(int i=1;i<numprocs;i++){
            all_flag_per_rank[i] = new int[num_step];
            for(int j=0;j<num_step;j++)all_flag_per_rank[i][j] = true;
        }
        /*int* flag_per_rank_1 = new int[num_step];
        for(int i=0;i<num_step;i++)flag_per_rank_1[i] = true;
        int* flag_per_rank_2 = new int[num_step];
        for(int i=0;i<num_step;i++)flag_per_rank_2[i] = true;
        int* flag_per_rank_3 = new int[num_step];
        for(int i=0;i<num_step;i++)flag_per_rank_3[i] = true;*/
        
        for(int i=1;i<numprocs;i++){
            MPI_Recv(all_flag_per_rank[i],num_step,MPI_INT,i,99,MPI_COMM_WORLD, &status);
        }
        /*MPI_Recv(flag_per_rank_1,num_step,MPI_INT,1,99,MPI_COMM_WORLD, &status);
        MPI_Recv(flag_per_rank_2,num_step,MPI_INT,2,99,MPI_COMM_WORLD, &status);
        MPI_Recv(flag_per_rank_3,num_step,MPI_INT,3,99,MPI_COMM_WORLD, &status);*/

        /*for(int i=0;i<num_step;i++){
            flag[i*numprocs+0]=flag_per_rank[i];
            flag[i*numprocs+1]=flag_per_rank_1[i];
            flag[i*numprocs+2]=flag_per_rank_2[i];
            flag[i*numprocs+3]=flag_per_rank_3[i];
        }*/
        for(int i=0;i<numprocs;i++){
            for(int j=0;j<num_step;j++){
                flag[j*numprocs+i]=all_flag_per_rank[i][j];
            }
        }
        gettimeofday(&te,NULL);
        std::cout<<"total time: "<<((te.tv_sec-ts.tv_sec)*1000+(te.tv_usec-ts.tv_usec)/1000)<<"ms"<<endl;
        for(int i=2;i<TotalNumStep;i++)if(flag[i])cout<<i<<" ";
        cout<<endl;
    }
    else{
        MPI_Send(flag_per_rank,num_step,MPI_INT,0,99,MPI_COMM_WORLD);
    }
    MPI_Finalize();
    return 0;
}


Overwriting lab/prime_mpi_omp.cpp


In [12]:
%pycat compile_prime.sh

#!/bin/bash
source /opt/intel/inteloneapi/setvars.sh > /dev/null 2>&1
/bin/echo "##" $(whoami) is compiling
mpiicpc -cxx=icpx lab/prime_mpi_omp.cpp -fiopenmp -fopenmp-targets=spir64 -o bin/prime_mpi.x


In [13]:
! chmod 755 q; chmod 755 compile_prime.sh; ./q compile_prime.sh nodes=1:ppn=2

Job has been submitted to Intel(R) DevCloud with  nodes and will execute soon.

 If you do not see result in 60 seconds, please restart the Jupyter kernel:
 Kernel -> 'Restart Kernel and Clear All Outputs...' and then try again

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2181640.v-qsvr-1           launch.sh        u175062         00:02:09 R batch          
2182606.v-qsvr-1           launch.sh        u175062         00:00:00 R batch          
2183280.v-qsvr-1           launch.sh        u175062         00:02:36 R batch          
2183625.v-qsvr-1           ...ub-singleuser u175062         00:00:05 R jupyterhub     
2183645.v-qsvr-1           compile_prime.sh u175062                0 Q batch          

Waiting for Output ██████████████████ Done⬇

########################################################################
#      Date:           Thu 09 Feb 2023 06:49:19 AM PST
#    Job I

In [14]:
%pycat launch.sh

#!/bin/bash
source /opt/intel/inteloneapi/setvars.sh > /dev/null 2>&1
/bin/echo "##" $(whoami) is executing
mpirun -np 4 bin/prime_mpi.x


In [15]:
! chmod 755 q; chmod 755 launch.sh; ./q launch.sh nodes=2:gpu:ppn=2

Job has been submitted to Intel(R) DevCloud with  nodes and will execute soon.

 If you do not see result in 60 seconds, please restart the Jupyter kernel:
 Kernel -> 'Restart Kernel and Clear All Outputs...' and then try again

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2181640.v-qsvr-1           launch.sh        u175062         00:02:09 R batch          
2182606.v-qsvr-1           launch.sh        u175062         00:00:00 R batch          
2183280.v-qsvr-1           launch.sh        u175062         00:02:36 R batch          
2183625.v-qsvr-1           ...ub-singleuser u175062         00:00:05 R jupyterhub     
2183648.v-qsvr-1           launch.sh        u175062                0 Q batch          

Waiting for Output ██████████ Done⬇

########################################################################
#      Date:           Thu 09 Feb 2023 06:49:37 AM PST
#    Job ID:      